<a href="https://colab.research.google.com/github/e2kk2e/datatool/blob/main/Kopie_von_create_zip_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os

def create_project_zip():
    zip_filename = "ProcureIntel_Complete_Kit.zip"

    # File contents definitions
    files = {
        # --- Root Configs ---
        "package.json": """{
  "name": "procurement-radar-api",
  "version": "0.1.0",
  "main": "src/index.js",
  "scripts": {
    "start": "node src/index.js",
    "dev": "nodemon src/index.js",
    "db:migrate": "node database/migrate.js"
  },
  "dependencies": {
    "express": "^4.18.2",
    "pg": "^8.11.3",
    "axios": "^1.6.2",
    "node-cron": "^3.0.3",
    "dotenv": "^16.3.1",
    "cors": "^2.8.5",
    "helmet": "^7.1.0"
  },
  "devDependencies": {
    "nodemon": "^3.0.1"
  }
}""",
        "docker-compose.yml": """version: '3.8'

services:
  api:
    build: .
    ports:
      - "3000:3000"
    environment:
      - NODE_ENV=development
      - DATABASE_URL=postgres://user:password@db:5432/procurement_radar
      - PORT=3000
    depends_on:
      - db
    command: >
      sh -c "npm run db:migrate && npm run dev"
    volumes:
      - .:/app
      - /app/node_modules

  db:
    image: postgres:15-alpine
    environment:
      POSTGRES_USER: user
      POSTGRES_PASSWORD: password
      POSTGRES_DB: procurement_radar
    ports:
      - "5432:5432"
    volumes:
      - pgdata:/var/lib/postgresql/data

volumes:
  pgdata:
""",
        "Dockerfile": """FROM node:20-alpine
WORKDIR /app
COPY package*.json ./
RUN npm ci --only=production
COPY . .
EXPOSE 3000
ENV NODE_ENV=production
ENV PORT=3000
CMD ["npm", "start"]
""",
        "cloudbuild.yaml": """steps:
  # 1. Build
  - name: 'gcr.io/cloud-builders/docker'
    args: ['build', '-t', 'europe-west3-docker.pkg.dev/$PROJECT_ID/procurement-repo/api:$COMMIT_SHA', '-t', 'europe-west3-docker.pkg.dev/$PROJECT_ID/procurement-repo/api:latest', '.']

  # 2. Push
  - name: 'gcr.io/cloud-builders/docker'
    args: ['push', 'europe-west3-docker.pkg.dev/$PROJECT_ID/procurement-repo/api:$COMMIT_SHA']

  # 3. Migrate DB
  - name: 'gcr.io/google.com/cloudsdktool/cloud-sdk'
    entrypoint: gcloud
    args:
      - 'run'
      - 'jobs'
      - 'execute'
      - 'migrate-job'
      - '--region=europe-west3'
      - '--wait'

  # 4. Deploy
  - name: 'gcr.io/google.com/cloudsdktool/cloud-sdk'
    entrypoint: gcloud
    args:
      - 'run'
      - 'deploy'
      - 'procurement-api'
      - '--image=europe-west3-docker.pkg.dev/$PROJECT_ID/procurement-repo/api:$COMMIT_SHA'
      - '--region=europe-west3'
      - '--platform=managed'
      - '--allow-unauthenticated'

images:
  - 'europe-west3-docker.pkg.dev/$PROJECT_ID/procurement-repo/api:$COMMIT_SHA'
  - 'europe-west3-docker.pkg.dev/$PROJECT_ID/procurement-repo/api:latest'
""",

        # --- Source Code ---
        "src/index.js": """require('dotenv').config();
const express = require('express');
const cors = require('cors');
const helmet = require('helmet');
const logger = require('./config/logger');

// Import routes
const tendersRoutes = require('./api/routes/tenders');
const analyticsRoutes = require('./api/routes/analytics');

const app = express();
const PORT = process.env.PORT || 3000;

app.use(helmet());
app.use(cors());
app.use(express.json());

// Routes
app.use('/api/tenders', tendersRoutes);
app.use('/api/analytics', analyticsRoutes);

app.get('/health', (req, res) => {
  res.json({ status: 'ok', service: 'procurement-radar' });
});

app.listen(PORT, () => {
  logger.info(`Server running on port ${PORT}`);
});
""",
        "src/config/logger.js": """const logger = {
  info: (msg, meta) => console.log(`[INFO] ${msg}`, meta || ''),
  error: (msg, meta) => console.error(`[ERROR] ${msg}`, meta || ''),
  debug: (msg, meta) => {
    if (process.env.NODE_ENV !== 'production') {
      console.log(`[DEBUG] ${msg}`, meta || '');
    }
  }
};
module.exports = logger;
""",
        "src/config/database.js": """const { Pool } = require('pg');
const logger = require('./logger');

const pool = new Pool({ connectionString: process.env.DATABASE_URL });

module.exports = {
  query: async (text, params) => {
    const start = Date.now();
    try {
      const res = await pool.query(text, params);
      const duration = Date.now() - start;
      logger.debug('executed query', { text, duration, rows: res.rowCount });
      return res;
    } catch (err) {
        logger.error('query error', { text, err });
        throw err;
    }
  },
  pool
};
""",
        "src/api/routes/tenders.js": """const express = require('express');
const router = express.Router();
const db = require('../../config/database');

// GET /api/tenders
router.get('/', async (req, res) => {
  try {
    const { region, cpv, limit = 20, offset = 0 } = req.query;

    let query = `
      SELECT id, source_id, source, title, cpv_codes, authority_name, estimated_value, deadline_date
      FROM tenders
      WHERE status = 'active'
    `;
    const params = [];
    let paramCount = 1;

    if (region) {
      query += ` AND region = $${paramCount}`;
      params.push(region);
      paramCount++;
    }

    if (cpv) {
      query += ` AND $${paramCount} = ANY(cpv_codes)`;
      params.push(cpv);
      paramCount++;
    }

    query += ` ORDER BY publication_date DESC LIMIT $${paramCount} OFFSET $${paramCount + 1}`;
    params.push(limit, offset);

    const result = await db.query(query, params);

    res.json({ success: true, count: result.rows.length, data: result.rows });
  } catch (err) {
    console.error(err);
    res.status(500).json({ error: 'Database error' });
  }
});

// GET /api/tenders/:id
router.get('/:id', async (req, res) => {
  try {
    const { id } = req.params;
    const result = await db.query('SELECT * FROM tenders WHERE id = $1', [id]);

    if (result.rows.length === 0) {
      return res.status(404).json({ error: 'Tender not found' });
    }

    res.json({ success: true, data: result.rows[0] });
  } catch (err) {
    res.status(500).json({ error: 'Server error' });
  }
});

module.exports = router;
""",
        "src/api/routes/analytics.js": """const express = require('express');
const router = express.Router();
const db = require('../../config/database');

router.get('/market-share', async (req, res) => {
  try {
    const query = `
      SELECT winner_name, COUNT(*) as win_count, SUM(award_value) as total_volume
      FROM awards
      WHERE award_date > NOW() - INTERVAL '1 year'
      GROUP BY winner_name
      ORDER BY total_volume DESC
      LIMIT 10
    `;
    const result = await db.query(query);
    res.json({ success: true, data: result.rows });
  } catch (err) {
    res.status(500).json({ error: 'Analytics error' });
  }
});

module.exports = router;
""",

        # --- Database ---
        "database/migrate.js": """require('dotenv').config();
const fs = require('fs');
const path = require('path');
const { Pool } = require('pg');

const pool = new Pool({ connectionString: process.env.DATABASE_URL });

async function migrate() {
  console.log('Starting migration...');
  const schemaPath = path.join(__dirname, 'schema.sql');
  const schema = fs.readFileSync(schemaPath, 'utf8');

  const client = await pool.connect();
  try {
    await client.query(schema);
    console.log('Schema applied successfully.');
  } catch (err) {
    console.error('Migration failed:', err);
  } finally {
    client.release();
    pool.end();
  }
}

migrate();
""",
        "database/schema.sql": """-- Users
CREATE TABLE IF NOT EXISTS users (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    api_key VARCHAR(64) UNIQUE,
    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
);

-- Tenders
CREATE TABLE IF NOT EXISTS tenders (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    source_id VARCHAR(255) NOT NULL,
    source VARCHAR(50) NOT NULL,
    title TEXT NOT NULL,
    description TEXT,
    cpv_codes TEXT[],
    authority_name VARCHAR(255),
    region VARCHAR(100),
    estimated_value DECIMAL(15, 2),
    publication_date DATE,
    deadline_date TIMESTAMP WITH TIME ZONE,
    status VARCHAR(50) DEFAULT 'active',
    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
    UNIQUE(source, source_id)
);

-- Awards
CREATE TABLE IF NOT EXISTS awards (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    tender_id UUID REFERENCES tenders(id),
    winner_name VARCHAR(255),
    award_value DECIMAL(15, 2),
    award_date DATE,
    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
);
""",

        # --- Terraform ---
        "terraform/main.tf": """terraform {
  required_providers {
    google = { source = "hashicorp/google", version = ">= 4.0" }
  }
}

provider "google" {
  project = var.project_id
  region  = var.region
}

resource "google_project_service" "apis" {
  for_each = toset(["run.googleapis.com", "sqladmin.googleapis.com", "artifactregistry.googleapis.com", "cloudbuild.googleapis.com", "secretmanager.googleapis.com"])
  service = each.key
  disable_on_destroy = false
}

resource "google_artifact_registry_repository" "repo" {
  location = var.region
  repository_id = "procurement-repo"
  format = "DOCKER"
  depends_on = [google_project_service.apis]
}

resource "google_sql_database_instance" "main" {
  name = "procurement-db-${random_id.db_suffix.hex}"
  database_version = "POSTGRES_15"
  region = var.region
  deletion_protection = false
  settings {
    tier = "db-f1-micro"
    availability_type = "ZONAL"
    ip_configuration { ipv4_enabled = true }
  }
  depends_on = [google_project_service.apis]
}

resource "random_id" "db_suffix" { byte_length = 4 }

resource "google_sql_database" "database" {
  name = "procurement_radar"
  instance = google_sql_database_instance.main.name
}

resource "google_sql_user" "users" {
  name = var.db_user
  instance = google_sql_database_instance.main.name
  password = var.db_password
}

resource "google_secret_manager_secret" "db_url" {
  secret_id = "DATABASE_URL"
  replication { auto {} }
  depends_on = [google_project_service.apis]
}

resource "google_secret_manager_secret_version" "db_url_val" {
  secret = google_secret_manager_secret.db_url.id
  secret_data = "postgres://${google_sql_user.users.name}:${var.db_password}@/procurement_radar?host=/cloudsql/${google_sql_database_instance.main.connection_name}"
}

resource "google_service_account" "app_sa" {
  account_id = "procurement-app-sa"
}

resource "google_project_iam_member" "sql_client" {
  project = var.project_id
  role = "roles/cloudsql.client"
  member = "serviceAccount:${google_service_account.app_sa.email}"
}

resource "google_secret_manager_secret_iam_member" "secret_access" {
  secret_id = google_secret_manager_secret.db_url.id
  role = "roles/secretmanager.secretAccessor"
  member = "serviceAccount:${google_service_account.app_sa.email}"
}

resource "google_cloud_run_v2_service" "default" {
  name = "procurement-api"
  location = var.region
  ingress = "INGRESS_TRAFFIC_ALL"
  template {
    service_account = google_service_account.app_sa.email
    containers {
      image = "europe-west3-docker.pkg.dev/${var.project_id}/procurement-repo/api:latest"
      env {
        name = "NODE_ENV"
        value = "production"
      }
      env {
        name = "DATABASE_URL"
        value_source {
          secret_key_ref {
            secret = google_secret_manager_secret.db_url.secret_id
            version = "latest"
          }
        }
      }
      volume_mounts {
        name = "cloudsql"
        mount_path = "/cloudsql"
      }
    }
    volumes {
      name = "cloudsql"
      cloud_sql_instance {
        instances = [google_sql_database_instance.main.connection_name]
      }
    }
  }
  depends_on = [google_project_service.apis]
  lifecycle {
    ignore_changes = [template[0].containers[0].image]
  }
}

resource "google_cloud_run_v2_job" "migrate" {
  name = "migrate-job"
  location = var.region
  template {
    template {
      service_account = google_service_account.app_sa.email
      containers {
        image = "europe-west3-docker.pkg.dev/${var.project_id}/procurement-repo/api:latest"
        command = ["npm", "run", "db:migrate"]
        env {
          name = "DATABASE_URL"
          value_source {
            secret_key_ref {
              secret = google_secret_manager_secret.db_url.secret_id
              version = "latest"
            }
          }
        }
        volume_mounts {
          name = "cloudsql"
          mount_path = "/cloudsql"
        }
      }
      volumes {
        name = "cloudsql"
        cloud_sql_instance {
          instances = [google_sql_database_instance.main.connection_name]
        }
      }
    }
  }
}
""",
        "terraform/variables.tf": """variable "project_id" {
  type = string
}
variable "region" {
  type = string
  default = "europe-west3"
}
variable "db_user" {
  type = string
  default = "radar_admin"
}
variable "db_password" {
  type = string
  sensitive = true
}
"""
    }

    # Write files to zip
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file_path, content in files.items():
            zipf.writestr(file_path, content)

    print(f"Created {zip_filename}")

if __name__ == "__main__":
    create_project_zip()

In [ ]:
from google.colab import userdata
userdata.get('GEMINI_API_KEY')

'AQ.Ab8RN6JEcExdmSx3eVFyrmhGShuv-Y1Gl2P2Qa0cMtTXWVDWdw'

# Task
Create the `src/services/collector.js` module to handle data ingestion. The module should:

1.  Import `axios` and the database pool from `../config/database`.
2.  Implement a `collectData` function that fetches tender data from a placeholder URL (e.g., "https://api.example.com/tenders").
3.  Normalize the data and insert it into the `tenders` table (handling duplicates via `ON CONFLICT` on `source` and `source_id`).
4.  If award data is present, insert it into the `awards` table.
5.  Export the `collectData` function.

## Implement Data Ingestion Service

### Subtask:
Create the `src/services/collector.js` module to handle fetching, normalizing, and storing tender data.


**Reasoning**:
The subtask requires creating a new directory and a JavaScript file for the data ingestion service. I will use Python's `os` module to create the directory and file I/O to write the specified content into `src/services/collector.js`.



In [ ]:
import os

# Ensure the services directory exists
os.makedirs('src/services', exist_ok=True)

# Define the content for src/services/collector.js
collector_js_content = """const axios = require('axios');
const db = require('../config/database');

const collectData = async () => {
  try {
    // Fetch data from placeholder URL
    const response = await axios.get('https://api.example.com/tenders');
    const items = response.data;

    for (const item of items) {
      // Insert into tenders table
      const query = `
        INSERT INTO tenders (
          source_id, source, title, cpv_codes, authority_name,
          region, estimated_value, publication_date, deadline_date
        ) VALUES ($1, $2, $3, $4, $5, $6, $7, $8, $9)
        ON CONFLICT (source, source_id)
        DO UPDATE SET
          title = EXCLUDED.title,
          updated_at = NOW()
        RETURNING id
      `;

      const values = [
        item.source_id,
        item.source || 'api',
        item.title,
        item.cpv_codes,
        item.authority_name,
        item.region,
        item.estimated_value,
        item.publication_date,
        item.deadline_date
      ];

      const result = await db.query(query, values);
      const tenderId = result.rows[0].id;

      // Insert Award if present
      if (item.award) {
        const awardQuery = `
          INSERT INTO awards (tender_id, winner_name, award_value, award_date)
          VALUES ($1, $2, $3, $4)
        `;
        await db.query(awardQuery, [
          tenderId,
          item.award.winner_name,
          item.award.award_value,
          item.award.award_date
        ]);
      }
    }
    console.log(`Processed ${items.length} items successfully.`);
  } catch (error) {
    console.error('Error in data collection:', error);
  }
};

module.exports = { collectData };
"""

# Write the file
with open('src/services/collector.js', 'w') as f:
    f.write(collector_js_content)

print("Successfully created src/services/collector.js")

Successfully created src/services/collector.js


## Setup Scheduler

### Subtask:
Update `src/index.js` to import the collector service and schedule it to run periodically using `node-cron`.


**Reasoning**:
Update `src/index.js` to include the `node-cron` scheduler and import the `collectData` service, then overwrite the file with the new content.



In [ ]:
index_js_content = """require('dotenv').config();
const express = require('express');
const cors = require('cors');
const helmet = require('helmet');
const cron = require('node-cron');
const logger = require('./config/logger');
const { collectData } = require('./services/collector');

// Import routes
const tendersRoutes = require('./api/routes/tenders');
const analyticsRoutes = require('./api/routes/analytics');

const app = express();
const PORT = process.env.PORT || 3000;

app.use(helmet());
app.use(cors());
app.use(express.json());

// Routes
app.use('/api/tenders', tendersRoutes);
app.use('/api/analytics', analyticsRoutes);

app.get('/health', (req, res) => {
  res.json({ status: 'ok', service: 'procurement-radar' });
});

// Schedule data collection every day at midnight
cron.schedule('0 0 * * *', () => {
  logger.info('Starting scheduled data collection');
  collectData();
});

app.listen(PORT, () => {
  logger.info(`Server running on port ${PORT}`);
});
"""

with open('src/index.js', 'w') as f:
    f.write(index_js_content)

print("Successfully updated src/index.js with scheduler configuration.")

Successfully updated src/index.js with scheduler configuration.
